In [17]:
#################################
# Raw FASTQ files preprocessing #
#################################
# 
# (Latest) library structure (may be changed):
# [Alu primer - 12 bp][Alu sequence - 6 bp][Flank][Adapter 1 - 10 bp][Adapter 2, barcode - 9 bp][Adapter3 - 12 bp]
# 
# R1: alu primer mate
# R2: adapter mate
# 
# 
# Steps:
# - separate reads into good and bad, depending on the mistake in the primers and the adapter
#     (as well as keeps barcodes good reads in the file for good R2 and location of mistake in the file bad R1)


# The number of permissible error:
mist = 1
# Primer, ad1 = Adapter 1, ad2 = Adapter 3 (aka Green)
primer = 'GAGCCACCGCGC'
ad1 = 'GCGTGCTGCGG'
ad2 = 'AGGGCGGT'
# Length of barcode
barlen = 9

# Input FASTQ files folder path.
inputdir = '~/data/'
# Output folder for processed FASTQ files.
outputdir = '~/data/processed'

# Check the following cell to verify the correctness of files

In [18]:
from Bio import SeqIO
import sys, argparse, os
import numpy as np
from os import listdir
from os.path import isfile, join

onlyfiles = [f for f in listdir(inputdir) if isfile(join(inputdir, f))]

if (len(onlyfiles) != 2):
	sys.exit("Error! Need only 2 files in directory: r1.fastq and r2.fastq")

filename1 = str(onlyfiles[0])
filename2 = str(onlyfiles[1])

print ('Trimming files: ' + filename1 + ', ' + filename2)

outputfile1, ext = os.path.splitext(filename1)
outputfile2, ext = os.path.splitext(filename2)

Trimming files: index7.R1.fastq, index7.R2.fastq


In [19]:
goodr1 = open(outputdir + outputfile1 + '_good.fastq', 'w')
goodr2 = open(outputdir + outputfile2 + '_good.fastq', 'w')
badr1 = open(outputdir + outputfile1 + '_bad.fastq', 'w')
badr2 = open(outputdir + outputfile2 + '_bad.fastq', 'w')

In [20]:
def hamming (x1, x2, m):
	j = 0
	for i in range(len(x1)):
		if x1[i] != x2[i]:
			j += 1
			if j > m:
				return (False)
	return (True)

def trim_primers(record, primer, m):
	record = str(record.format('fastq'))
	record = record.split('\n')
	record = record[1]
	badgood = {'good':False, 'bad':np.array([0, 0, 0])}
	len_primer = len(primer)
	ham_prim = (hamming(primer, record[0:len_primer], m))or(hamming(primer, record[1:(len_primer+1)], m))
	if ham_prim:
		badgood['good'] = True
	else:
		badgood['bad'] = np.array([1, 0, 0])
	return badgood

def trim_ads(record, ad1, ad2, barlen, m):
	record = str(record.format('fastq'))
	record = record.split('\n')
	record = record[1]
	badgood = {'good':False, 'bad':np.array([0, 0, 0]), 'barcode':None}
	len_ad1 = len(ad1)
	len_ad2 = len(ad2)
	seq1 = record[0:len_ad1]
	seq1_shift = record[1:(len_ad1+1)]
	seq2 = record[(len_ad1+barlen):(len_ad1+barlen+len_ad2)]
	seq2_shift = record[(len_ad1+barlen+1):(len_ad1+barlen+len_ad2+1)]
	ham_ad1 = (hamming(ad1, seq1, m))or(hamming(ad1, seq1_shift, m))
	ham_ad2 = (hamming(ad2, seq2, m))or(hamming(ad1, seq2_shift, m))
	if (ham_ad1)and(ham_ad2):
		badgood['good'] = True
		if hamming(ad2, seq2, m) > hamming(ad1, seq2_shift, m):
			badgood['barcode'] = record[len_ad1:(len_ad1+barlen)]
		else: badgood['barcode'] = record[(len_ad1+1):(len_ad1+barlen+1)]
	else:
		if not((ham_ad1)or(ham_ad2)):
			badgood['bad'] = np.array([0, 1, 1])
		elif not(ham_ad1):
			badgood['bad'] = np.array([0, 1, 0])
		else:
			badgood['bad'] = np.array([0, 0, 1])
	return badgood

def concate(x1, x2):
	result = ''
	for i in range(len(x1)):
		result = result + str(x1[i]) + '-' + str(x2[i])
		if (i != (len(x1) - 1)): result = result + ','
	return (result)

In [21]:
original_R1_reads = SeqIO.parse(inputdir + filename1, "fastq")
original_R2_reads = SeqIO.parse(inputdir + filename2, "fastq")

In [22]:
count = np.array([0, 0, 0])
elem = ('primer', 'ad', 'green')

for zipi in zip(original_R1_reads, original_R2_reads):
	r1,r2 = zipi
	fr1 = trim_primers(r1, primer, mist)
	if fr1['good']:
		fr2 = trim_ads(r2, ad1, ad2, barlen, mist)
		if fr2['good']:
			goodread = str(r2.format('fastq'))
			goodread = goodread.split('\n')
			goodread[0] = goodread[0] + ' barcode:' + str(fr2['barcode'])
			goodread = '\n'.join(goodread)
			goodr1.write(str(r1.format('fastq')))
			goodr2.write(goodread)
		else:
			badread = str(r1.format('fastq'))
			badread = badread.split('\n')
			badread[0] = badread[0] + ' reason:' + concate(elem, (np.char.mod('%d', fr2['bad'])))
			badread = '\n'.join(badread)
			badr1.write(badread)
			badr2.write(str(r2.format('fastq')))
			count = np.sum([count, fr2['bad']], axis=0)
	else:
		badread = str(r1.format('fastq'))
		badread = badread.split('\n')
		badread[0] = badread[0] + ' reason:' + concate(elem, (np.char.mod('%d', fr1['bad'])))
		badread = '\n'.join(badread)
		badr1.write(badread)
		badr2.write(str(r2.format('fastq')))
		count = np.sum([count, fr1['bad']], axis=0)

In [23]:
goodr1.close()
goodr2.close()
badr1.close()
badr2.close()

In [24]:
print (count)

[31 20 40]
